<a href="https://colab.research.google.com/github/TheophilusG/DataBootcamp/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas scikit-learn

In [ ]:
import pandas as pd

# Load the datasets
train_data = pd.read_csv('/content/train data.csv')
test_data = pd.read_csv('/content/test data.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

def preprocess_data(data, encoders=None, scaler=None, is_train=True):
    categorical_columns = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
    numerical_columns = ['person_age', 'person_income', 'person_emp_length',
                         'loan_amnt', 'loan_int_rate', 'loan_percent_income',
                         'cb_person_cred_hist_length']

    # Encode categorical variables
    if is_train:
        encoders = {col: LabelEncoder().fit(data[col]) for col in categorical_columns}
    for col in categorical_columns:
        data[col] = encoders[col].transform(data[col])

    # Scale numerical variables
    if is_train:
        scaler = StandardScaler().fit(data[numerical_columns])
    data[numerical_columns] = scaler.transform(data[numerical_columns])

    return data, encoders, scaler

# Split training data into features and target
X_train = train_data.drop(columns=['id', 'loan_status'])
y_train = train_data['loan_status']

# Preprocess training data
X_train, encoders, scaler = preprocess_data(X_train, is_train=True)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

# Train-test split for evaluation
from sklearn.model_selection import train_test_split
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Train the model
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
rf_model.fit(X_train_split, y_train_split)

# Evaluate precision on the validation set
y_val_pred = rf_model.predict(X_val_split)
print(f"Precision on validation set: {precision_score(y_val_split, y_val_pred):.4f}")

Precision on validation set: 0.9371


In [ ]:
# Preprocess test data
X_test = test_data.drop(columns=['id'])
X_test, _, _ = preprocess_data(X_test, encoders=encoders, scaler=scaler, is_train=False)

In [ ]:
# Predict on the test data
test_predictions = rf_model.predict(X_test)

# Create the submission DataFrame
submission = test_data[['id']].copy()
submission['prediction'] = test_predictions

# Save the submission file
submission.to_csv("submission.csv", index=False)
print("Submission file saved as 'submission.csv'")

Submission file saved as 'submission.csv'
